In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

In [4]:
df1 = pd.read_csv('Stable_Data_CSV.csv')
df2 = pd.read_csv('6wUsingBL.csv')
df_stable = df2[df2.set_index(['participant_id']).index.isin(df1.set_index(['participant_id']).index)]
df_stable_US = df_stable.loc[(df_stable['arm'] == 1)]
df_stable_WN = df_stable.loc[(df_stable['arm'] == 2)]
print(len(df_stable_US))
print(len(df_stable_WN))
df_stable_US=(df_stable_US[['c_3','sl_1','a_2','q_1','tfi_total.1','tfi_total.2','tfi_total.3']])
df_stable_WN=(df_stable_WN[['sl_2','r_1','e_1','tfi_total.1','tfi_total.2','tfi_total.3']])

31
30


In [31]:
def data_prep(df):
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    train_data=df.iloc[:-5,:]
    test_data =df.iloc[-5:,:]
    #Train Test Split 
    x_train=train_data.drop(['tfi_total.2','tfi_total.3'],axis=1)
    y_train=train_data[['tfi_total.2']]
    x_test=test_data.drop(['tfi_total.2','tfi_total.3'],axis=1)
    y_test=test_data[['tfi_total.2']]
    sc_X = sc_X.fit(x_train)
    sc_y = sc_y.fit(y_train)
    x_train=sc_X.transform(x_train)
    x_test=sc_X.transform(x_test)
    y_train=sc_y.transform(y_train)
    y_test=sc_y.transform(y_test)

    print(x_train.shape,y_train.shape, x_test.shape, y_test.shape)
    return x_train, x_test, y_train, y_test,sc_y

US_x_train, US_x_test, US_y_train, US_y_test,US_sc_y=data_prep(df_stable_US)
WN_x_train, WN_x_test, WN_y_train, WN_y_test,WN_sc_y=data_prep(df_stable_WN)

US_regressor = SVR(kernel = 'rbf')
WN_regressor = SVR(kernel = 'rbf')
US_regressor.fit(US_x_train, US_y_train)
WN_regressor.fit(WN_x_train, WN_y_train)

(26, 5) (26, 1) (5, 5) (5, 1)
(25, 4) (25, 1) (5, 4) (5, 1)


/Users/manjuvallayil/miniforge3/envs/tinnitus/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/manjuvallayil/miniforge3/envs/tinnitus/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [32]:
US_y_test=US_sc_y.inverse_transform( US_y_test.reshape(-1, 1)) 
US_yhat = US_regressor.predict(US_x_test)
US_yhat  = US_sc_y.inverse_transform(US_yhat.reshape(-1, 1)) 
WN_y_test=WN_sc_y.inverse_transform( WN_y_test.reshape(-1, 1)) 
WN_yhat = WN_regressor.predict(WN_x_test)
WN_yhat  = WN_sc_y.inverse_transform(WN_yhat.reshape(-1, 1)) 

In [34]:
US_error=mean_absolute_error(US_y_test,US_yhat)
print("US_mae")
print(US_error)
WN_error=mean_absolute_error(WN_y_test,WN_yhat)
print("\n")
print("WN_mae")
print(WN_error)

US_mae
15.420003098447319


WN_mae
13.598193903510042
